In [1]:
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
from imutils import face_utils
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import threading
import sounddevice as sd
import soundfile as sf
import pygame
import tempfile

# Configuration des chemins
video_dir = "../data/video_recordings"
lip_dir = "../data/lip_frames"
audio_dir = "../data/audio_recordings"  # Nouveau dossier pour les fichiers audio

# Mots cibles à enregistrer
target_words = ["oui", "non", "un", "deux"]

# Nombre d'exemples par mot
examples_per_word = 10

# Paramètres d'enregistrement
frame_rate = 30    # Images par seconde
duration = 1       # Durée d'enregistrement en secondes (1 seconde suffit pour les mots courts)
width = 640        # Largeur de la vidéo
height = 480       # Hauteur de la vidéo
audio_fs = 16000   # Fréquence d'échantillonnage audio (Hz)

# Créer les dossiers s'ils n'existent pas
for word in target_words:
    word_video_dir = os.path.join(video_dir, word)
    word_lip_dir = os.path.join(lip_dir, word)
    word_audio_dir = os.path.join(audio_dir, word)
    os.makedirs(word_video_dir, exist_ok=True)
    os.makedirs(word_lip_dir, exist_ok=True)
    os.makedirs(word_audio_dir, exist_ok=True)

# Initialisation du détecteur de visage et du prédicteur de points de repère
detector = dlib.get_frontal_face_detector()
predictor_path = "../models/shape_predictor_68_face_landmarks.dat"
if os.path.exists(predictor_path):
    predictor = dlib.shape_predictor(predictor_path)
    print("Détecteur de points de repère chargé !")
else:
    print(f"ATTENTION: Le fichier {predictor_path} n'a pas été trouvé.")
    print("Téléchargez-le depuis http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")

pygame 2.6.1 (SDL 2.28.4, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
Détecteur de points de repère chargé !


# Détection et extraction des régions des lèvres pour la VSR

La détection et l'extraction des régions des lèvres constituent la base technique de tout système de reconnaissance visuelle de la parole (VSR). Ce document détaille l'implémentation et le fonctionnement de ces processus dans le code.

## Détection des points du visage

```python
# Initialisation du détecteur de visage et du prédicteur de points de repère
detector = dlib.get_frontal_face_detector()
predictor_path = "../models/shape_predictor_68_face_landmarks.dat"
if os.path.exists(predictor_path):
    predictor = dlib.shape_predictor(predictor_path)
```

Le système repose sur deux composants principaux de dlib:
1. **Détecteur de visage frontal**: Identifie les visages dans l'image
2. **Prédicteur de points de repère**: Modèle pré-entraîné qui localise 68 points caractéristiques du visage

## Fonctionnement du shape_predictor

Le modèle `shape_predictor_68_face_landmarks.dat` est basé sur une architecture d'apprentissage "Ensemble of Regression Trees" (ERT). Ce modèle a été entraîné sur des milliers d'images de visages annotées manuellement et peut prédire la position des 68 points faciaux avec grande précision, même dans des conditions variables.

## Les 68 points faciaux et leur signification

Le modèle utilise une convention standard de 68 points disposés ainsi:
* Points 1-17 : Contour du visage
* Points 18-22 : Sourcil gauche
* Points 23-27 : Sourcil droit
* Points 28-36 : Nez
* Points 37-42 : Œil gauche
* Points 43-48 : Œil droit
* **Points 49-68 : Région buccale**

Pour la VSR, nous utilisons spécifiquement:
* **Points 49-60 : Contour extérieur des lèvres**
* **Points 61-68 : Contour intérieur des lèvres**


In [2]:
# Initialisation de pygame pour la lecture audio
pygame.mixer.init()

In [1]:
def extract_lip_region(frame):
    """
    Extrait la région des lèvres d'une image.
    
    Args:
        frame: Image à analyser
    
    Returns:
        tuple: (region des lèvres, points des lèvres, visage détecté)
    """
    try:
        # Convertir en niveaux de gris pour la détection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Détecter les visages
        faces = detector(gray, 0)
        
        if len(faces) == 0:
            return None, None, None
        
        # Prendre le premier visage détecté
        face = faces[0]
        
        # Prédire les points de repère
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)
        
        # Les points des lèvres sont les points 48-68 dans le modèle à 68 points
        lips_points = shape[48:68]
        
        # Calculer le rectangle englobant pour les lèvres
        x, y = lips_points.min(axis=0)
        w, h = lips_points.max(axis=0) - lips_points.min(axis=0)
        
        # Ajouter une marge
        margin = 10
        x = max(0, x - margin)
        y = max(0, y - margin)
        w = min(frame.shape[1] - x, w + 2 * margin)
        h = min(frame.shape[0] - y, h + 2 * margin)
        
        # Extraire la région des lèvres
        lip_region = frame[y:y+h, x:x+w]
        
        return lip_region, lips_points, face
    except Exception as e:
        print(f"Erreur lors de l'extraction des lèvres: {e}")
        return None, None, None

## Analyse étape par étape

### 1. Préparation de l'image
```python
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
```
Cette conversion transforme l'image du format BGR (OpenCV) en niveaux de gris.

### 2. Détection du visage
```python
faces = detector(gray, 0)
if len(faces) == 0:
    return None, None, None
face = faces[0]
```
Le détecteur utilise un algorithme HOG+SVM pour localiser les visages. Le paramètre `0` indique qu'aucune pyramide d'image n'est utilisée (pas d'upsampling). Si aucun visage n'est détecté, la fonction retourne immédiatement `None`.

### 3. Détection des points de repère
```python
shape = predictor(gray, face)
shape = face_utils.shape_to_np(shape)
```
Le prédicteur analyse la région faciale et génère 68 points. La fonction `shape_to_np` convertit l'objet dlib en tableau NumPy.

### 4. Isolation des points des lèvres
```python
lips_points = shape[48:68]
```
Seuls les 20 points correspondant aux lèvres sont conservés.

### 5. Calcul du rectangle englobant
```python
x, y = lips_points.min(axis=0)
w, h = lips_points.max(axis=0) - lips_points.min(axis=0)
```
Cette opération trouve les coordonnées extrêmes des points pour créer un rectangle contenant tous les points des lèvres.

### 6. Ajout d'une marge
```python
margin = 10
x = max(0, x - margin)
y = max(0, y - margin)
w = min(frame.shape[1] - x, w + 2 * margin)
h = min(frame.shape[0] - y, h + 2 * margin)
```
Une marge de 10 pixels est ajoutée autour des lèvres pour s'assurer que toute la région est capturée. Les conditions `max()` et `min()` garantissent que le rectangle reste dans les limites de l'image.

### 7. Extraction de la région
```python
lip_region = frame[y:y+h, x:x+w]
```
Cette opération crée une sous-image contenant uniquement la région des lèvres.

## Visualisation des points des lèvres

Pour le retour visuel pendant la capture, nous dessinons les points détectés:

```python
if face is not None and lips_points is not None:
    # Dessiner les points des lèvres
    for (x, y) in lips_points:
        cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)
```

Cela permet de confirmer visuellement que la détection fonctionne correctement.

## Évaluation de la qualité de détection

Le système évalue la qualité de l'enregistrement en calculant le pourcentage de frames où les lèvres ont été correctement détectées:

```python
if len(self.lip_regions) < len(self.frames) * 0.7:
    self.video_quality.set("Mauvais: Lèvres détectées dans moins de 70% des images")
elif len(self.lip_regions) < len(self.frames) * 0.9:
    self.video_quality.set("Acceptable: Lèvres détectées dans plus de 70% des images")
else:
    self.video_quality.set("Excellent: Lèvres détectées dans plus de 90% des images")
```


## Prétraitement additionnel pour la VSR

Dans un système VSR complet, les régions extraites subissent généralement des traitements supplémentaires:

1. **Redimensionnement**: Normalisation à une taille standard (par exemple 64×64 pixels)
2. **Normalisation d'intensité**: Ajustement des niveaux de luminosité et contraste
3. **Conversion en niveaux de gris**: Réduction de la dimensionnalité
4. **Augmentation de données**: Rotation, mise à l'échelle, et variations d'intensité pour améliorer la robustesse

Ces étapes seront incorporées dans la phase d'entraînement du modèle VSR.

In [4]:
class VSRDataCollector:
    def __init__(self, root):
        self.root = root
        self.root.title("Collecteur de données VSR")
        self.root.geometry("1000x700")
        self.root.resizable(True, True)
        
        # Variables
        self.cap = None
        self.is_recording = False
        self.frames = []
        self.lip_regions = []
        self.audio_data = None
        self.current_word = tk.StringVar(value=target_words[0])
        self.current_example = tk.IntVar(value=1)
        self.countdown_value = tk.IntVar(value=3)
        self.progress = tk.DoubleVar(value=0)
        self.recording_thread = None
        self.video_quality = tk.StringVar(value="")
        self.temp_audio_file = None
        self.temp_video_file = None
        self.status_message = tk.StringVar(value="")  # Nouvelle variable pour les messages
        
        # Configurer l'interface
        self.setup_ui()
        
        # Démarrer la webcam
        self.start_webcam()
        
        # Configurer la fermeture propre
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
    
    def setup_ui(self):
        # Créer une structure de grille
        main_frame = ttk.Frame(self.root, padding=10)
        main_frame.grid(row=0, column=0, sticky="nsew")
        
        # Configurer les poids des lignes et colonnes
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        main_frame.columnconfigure(0, weight=2)
        main_frame.columnconfigure(1, weight=1)
        main_frame.rowconfigure(0, weight=1)
        
        # Créer les deux panneaux principaux
        left_panel = ttk.Frame(main_frame, padding=5)
        left_panel.grid(row=0, column=0, sticky="nsew")
        
        right_panel = ttk.Frame(main_frame, padding=5)
        right_panel.grid(row=0, column=1, sticky="nsew")
        
        # Configurer le panneau gauche (vidéo)
        left_panel.columnconfigure(0, weight=1)
        left_panel.rowconfigure(0, weight=1)
        left_panel.rowconfigure(1, weight=0)
        
        # Webcam view
        self.webcam_label = ttk.Label(left_panel, borderwidth=2, relief="solid")
        self.webcam_label.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)
        
        # Barre de progression
        progress_frame = ttk.Frame(left_panel)
        progress_frame.grid(row=1, column=0, sticky="ew", padx=5, pady=5)
        
        self.progress_bar = ttk.Progressbar(progress_frame, orient="horizontal", mode="determinate", variable=self.progress)
        self.progress_bar.pack(fill="x", expand=True)
        
        # Configurer le panneau droit (contrôles)
        right_panel.columnconfigure(0, weight=1)
        for i in range(10):  # Ajout d'une ligne pour le statut
            right_panel.rowconfigure(i, weight=0)
        right_panel.rowconfigure(10, weight=1)  # Pour la zone de résultats
        
        # Titre
        title_label = ttk.Label(right_panel, text="Collecte de données VSR", font=("Arial", 16, "bold"))
        title_label.grid(row=0, column=0, sticky="ew", padx=5, pady=10)
        
        # Sélection du mot
        word_frame = ttk.LabelFrame(right_panel, text="Mot à prononcer")
        word_frame.grid(row=1, column=0, sticky="ew", padx=5, pady=5)
        
        word_display = ttk.Label(word_frame, textvariable=self.current_word, font=("Arial", 40, "bold"))
        word_display.pack(padx=20, pady=20)
        
        # Numéro d'exemple
        example_frame = ttk.Frame(right_panel)
        example_frame.grid(row=2, column=0, sticky="ew", padx=5, pady=5)
        
        example_label = ttk.Label(example_frame, text="Exemple:")
        example_label.pack(side="left", padx=5)
        
        example_counter = ttk.Label(example_frame, textvariable=self.current_example)
        example_counter.pack(side="left", padx=5)
        
        ttk.Label(example_frame, text="/").pack(side="left")
        
        ttk.Label(example_frame, text=str(examples_per_word)).pack(side="left", padx=5)
        
        # Boutons de navigation
        nav_frame = ttk.Frame(right_panel)
        nav_frame.grid(row=3, column=0, sticky="ew", padx=5, pady=5)
        
        prev_word_btn = ttk.Button(nav_frame, text="Mot précédent", command=self.prev_word)
        prev_word_btn.pack(side="left", padx=5, expand=True)
        
        next_word_btn = ttk.Button(nav_frame, text="Mot suivant", command=self.next_word)
        next_word_btn.pack(side="left", padx=5, expand=True)
        
        # Boutons d'enregistrement
        record_frame = ttk.Frame(right_panel)
        record_frame.grid(row=4, column=0, sticky="ew", padx=5, pady=10)
        
        self.record_btn = ttk.Button(record_frame, text="Enregistrer", command=self.start_recording)
        self.record_btn.pack(fill="x", padx=5, pady=5)
        
        # Compte à rebours
        countdown_frame = ttk.Frame(right_panel)
        countdown_frame.grid(row=5, column=0, sticky="ew", padx=5, pady=5)
        
        self.countdown_label = ttk.Label(countdown_frame, textvariable=self.countdown_value, font=("Arial", 30))
        self.countdown_label.pack(padx=5, pady=5)
        
        # Qualité de l'enregistrement
        quality_frame = ttk.LabelFrame(right_panel, text="Qualité de l'enregistrement")
        quality_frame.grid(row=6, column=0, sticky="ew", padx=5, pady=5)
        
        quality_label = ttk.Label(quality_frame, textvariable=self.video_quality, font=("Arial", 12))
        quality_label.pack(padx=5, pady=5)
        
        # Bouton de replay
        replay_frame = ttk.Frame(right_panel)
        replay_frame.grid(row=7, column=0, sticky="ew", padx=5, pady=5)
        
        self.replay_btn = ttk.Button(replay_frame, text="Rejouer avec son", command=self.replay_recording, state="disabled")
        self.replay_btn.pack(fill="x", padx=5, pady=5)
        
        # Boutons de validation
        validation_frame = ttk.Frame(right_panel)
        validation_frame.grid(row=8, column=0, sticky="ew", padx=5, pady=5)
        
        accept_btn = ttk.Button(validation_frame, text="Accepter", command=self.accept_recording)
        accept_btn.pack(side="left", padx=5, expand=True)
        
        reject_btn = ttk.Button(validation_frame, text="Rejeter", command=self.reject_recording)
        reject_btn.pack(side="left", padx=5, expand=True)
        
        # Zone de statut (nouveau)
        status_frame = ttk.Frame(right_panel)
        status_frame.grid(row=9, column=0, sticky="ew", padx=5, pady=5)
        
        status_label = ttk.Label(status_frame, textvariable=self.status_message, font=("Arial", 10), wraplength=250)
        status_label.pack(fill="x", padx=5, pady=5)
        
        # Zone de résultats
        results_frame = ttk.LabelFrame(right_panel, text="Résultats de détection")
        results_frame.grid(row=10, column=0, sticky="nsew", padx=5, pady=5)
        
        self.lip_canvas = tk.Canvas(results_frame, bg="white")
        self.lip_canvas.pack(fill="both", expand=True, padx=5, pady=5)
    
    def start_webcam(self):
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        self.update_webcam()
    
    def update_webcam(self):
        if self.cap is not None and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                # Dessiner un rectangle au milieu pour guider l'utilisateur
                h, w = frame.shape[:2]
                center_x, center_y = w // 2, h // 2
                rect_w, rect_h = 300, 200
                cv2.rectangle(frame, 
                             (center_x - rect_w // 2, center_y - rect_h // 2),
                             (center_x + rect_w // 2, center_y + rect_h // 2),
                             (0, 255, 0), 2)
                
                # Extraction des lèvres pour le retour en direct (facultatif)
                if not self.is_recording:
                    lip_region, lips_points, face = extract_lip_region(frame)
                    if face is not None and lips_points is not None:
                        # Dessiner les points des lèvres
                        for (x, y) in lips_points:
                            cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)
                
                # Convertir l'image pour l'affichage Tkinter
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame_rgb)
                imgtk = ImageTk.PhotoImage(image=img)
                
                # Mettre à jour l'affichage
                self.webcam_label.imgtk = imgtk
                self.webcam_label.configure(image=imgtk)
            
            # Mettre à jour périodiquement
            self.root.after(15, self.update_webcam)
    
    def prev_word(self):
        if not self.is_recording:
            current_idx = target_words.index(self.current_word.get())
            if current_idx > 0:
                self.current_word.set(target_words[current_idx - 1])
                self.current_example.set(1)
    
    def next_word(self):
        if not self.is_recording:
            current_idx = target_words.index(self.current_word.get())
            if current_idx < len(target_words) - 1:
                self.current_word.set(target_words[current_idx + 1])
                self.current_example.set(1)
    
    def start_recording(self):
        if not self.is_recording and self.cap is not None:
            self.record_btn.configure(state="disabled")
            self.replay_btn.configure(state="disabled")
            self.is_recording = True
            self.frames = []
            self.lip_regions = []
            self.audio_data = None
            self.progress.set(0)
            self.status_message.set("")  # Effacer le message de statut
            
            # Nettoyer les fichiers temporaires précédents
            self.cleanup_temp_files()
            
            # Démarrer l'enregistrement dans un thread séparé
            self.recording_thread = threading.Thread(target=self.recording_process)
            self.recording_thread.daemon = True
            self.recording_thread.start()
    
    def recording_process(self):
        # Initialiser le compte à rebours
        for i in range(3, 0, -1):
            self.countdown_value.set(i)
            self.root.update()
            time.sleep(0.7)
        
        self.countdown_value.set("GO!")
        self.root.update()
        
        # Configurer l'enregistrement audio
        audio_frames = []
        
        # Fonction de callback pour l'enregistrement audio
        def audio_callback(indata, frames, time, status):
            if status:
                print(f"Erreur audio: {status}")
            audio_frames.append(indata.copy())
        
        # Démarrer l'enregistrement audio dans un thread
        audio_stream = sd.InputStream(samplerate=audio_fs, channels=1, callback=audio_callback)
        audio_stream.start()
        
        # Enregistrement vidéo
        start_time = time.time()
        frame_count = 0
        max_frames = int(duration * frame_rate)
        
        while time.time() - start_time < duration:
            if self.cap is not None:
                ret, frame = self.cap.read()
                if ret:
                    self.frames.append(frame)
                    
                    # Extraire et sauvegarder la région des lèvres
                    lip_region, _, _ = extract_lip_region(frame)
                    if lip_region is not None:
                        self.lip_regions.append(lip_region)
                    
                    # Mise à jour de la progression
                    frame_count += 1
                    progress_percent = min(100, (frame_count / max_frames) * 100)
                    self.progress.set(progress_percent)
                    
                    # Petite pause pour ne pas surcharger
                    time.sleep(0.01)
        
        # Arrêter l'enregistrement audio
        audio_stream.stop()
        audio_stream.close()
        
        # Convertir et sauvegarder l'audio dans un fichier temporaire
        if audio_frames:
            audio_data = np.vstack(audio_frames)
            self.audio_data = audio_data.flatten()
            
            # Créer un fichier temporaire pour l'audio
            fd, temp_path = tempfile.mkstemp(suffix='.wav')
            os.close(fd)
            self.temp_audio_file = temp_path
            sf.write(self.temp_audio_file, self.audio_data, audio_fs)
        
        # Sauvegarder la vidéo dans un fichier temporaire pour le replay
        if self.frames:
            fd, temp_path = tempfile.mkstemp(suffix='.mp4')
            os.close(fd)
            self.temp_video_file = temp_path
            
            # Taille de la première image
            h, w = self.frames[0].shape[:2]
            
            # Créer l'objet VideoWriter
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(self.temp_video_file, fourcc, frame_rate, (w, h))
            
            # Écrire chaque image dans le fichier vidéo
            for frame in self.frames:
                out.write(frame)
            
            # Libérer les ressources
            out.release()
        
        # Terminer l'enregistrement
        self.countdown_value.set("Fin")
        self.is_recording = False
        
        # Analyser la qualité de l'enregistrement
        self.analyze_recording()
        
        # Réactiver les boutons
        self.record_btn.configure(state="normal")
        if self.temp_audio_file and self.temp_video_file:
            self.replay_btn.configure(state="normal")
            self.status_message.set("Enregistrement terminé. Vous pouvez rejouer, accepter ou rejeter.")
    
    def replay_recording(self):
        """
        Rejoue l'enregistrement avec le son
        """
        if not self.temp_audio_file:
            self.status_message.set("Pas d'audio disponible pour le replay!")
            return
        
        # Lecture du son
        try:
            pygame.mixer.music.load(self.temp_audio_file)
            pygame.mixer.music.play()
            
            # Afficher des informations dans la zone de statut
            self.status_message.set("Lecture de l'audio en cours...")
            
            # Désactiver les boutons pendant la lecture
            self.replay_btn.configure(state="disabled")
            
            # Vérifier périodiquement si la lecture est terminée
            def check_playback_done():
                if pygame.mixer.music.get_busy():
                    self.root.after(100, check_playback_done)
                else:
                    self.replay_btn.configure(state="normal")
                    self.status_message.set("Lecture terminée")
            
            check_playback_done()
            
        except Exception as e:
            self.status_message.set(f"Erreur lors de la lecture: {e}")
            self.replay_btn.configure(state="normal")
    
    def cleanup_temp_files(self):
        """
        Nettoie les fichiers temporaires
        """
        if self.temp_audio_file and os.path.exists(self.temp_audio_file):
            try:
                os.remove(self.temp_audio_file)
            except:
                pass
            self.temp_audio_file = None
        
        if self.temp_video_file and os.path.exists(self.temp_video_file):
            try:
                os.remove(self.temp_video_file)
            except:
                pass
            self.temp_video_file = None
    
    def analyze_recording(self):
        if not self.frames:
            self.video_quality.set("Aucune image capturée!")
            return
        
        # Analyse simple: vérifier que des lèvres ont été détectées
        if len(self.lip_regions) < len(self.frames) * 0.7:
            self.video_quality.set("Mauvais: Lèvres détectées dans moins de 70% des images")
        elif len(self.lip_regions) < len(self.frames) * 0.9:
            self.video_quality.set("Acceptable: Lèvres détectées dans plus de 70% des images")
        else:
            self.video_quality.set("Excellent: Lèvres détectées dans plus de 90% des images")
        
        # Afficher quelques images des lèvres dans le canvas
        self.show_lip_results()
    
    def show_lip_results(self):
        if not self.lip_regions:
            return
        
        # Effacer le canvas
        self.lip_canvas.delete("all")
        
        # Sélectionner quelques images à afficher
        num_display = min(6, len(self.lip_regions))
        display_indices = np.linspace(0, len(self.lip_regions)-1, num_display, dtype=int)
        
        # Calculer la taille et la position des images
        canvas_width = self.lip_canvas.winfo_width()
        canvas_height = self.lip_canvas.winfo_height()
        
        if canvas_width <= 1 or canvas_height <= 1:
            # Le canvas n'est pas encore initialisé, attendre et réessayer
            self.root.after(100, self.show_lip_results)
            return
        
        img_width = (canvas_width - 20) // 3
        img_height = (canvas_height - 20) // 2
        
        for i, idx in enumerate(display_indices):
            # Position dans une grille 3x2
            row = i // 3
            col = i % 3
            x = 10 + col * img_width
            y = 10 + row * img_height
            
            # Obtenir et redimensionner l'image des lèvres
            lip_img = self.lip_regions[idx].copy()
            lip_img = cv2.resize(lip_img, (img_width - 10, img_height - 10))
            lip_img_rgb = cv2.cvtColor(lip_img, cv2.COLOR_BGR2RGB)
            
            # Convertir en format Tkinter
            pil_img = Image.fromarray(lip_img_rgb)
            tk_img = ImageTk.PhotoImage(pil_img)
            
            # Sauvegarder la référence (important pour éviter le garbage collection)
            self.lip_canvas.imgs = getattr(self.lip_canvas, 'imgs', [])
            self.lip_canvas.imgs.append(tk_img)
            
            # Afficher l'image
            self.lip_canvas.create_image(x, y, anchor="nw", image=tk_img)
            self.lip_canvas.create_text(x + img_width//2, y + img_height - 10, 
                                       text=f"Frame {idx}", fill="black")
    
    def accept_recording(self):
        if not self.frames or not self.lip_regions:
            self.status_message.set("Aucun enregistrement à sauvegarder!")
            return
        
        word = self.current_word.get()
        example_idx = self.current_example.get()
        
        # Sauvegarder la vidéo
        video_filename = f"{word}_{example_idx:02d}.mp4"
        video_path = os.path.join(video_dir, word, video_filename)
        
        # Taille de la première image
        h, w = self.frames[0].shape[:2]
        
        # Créer l'objet VideoWriter
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(video_path, fourcc, frame_rate, (w, h))
        
        # Écrire chaque image dans le fichier vidéo
        for frame in self.frames:
            out.write(frame)
        
        # Libérer les ressources
        out.release()
        
        # Sauvegarder l'audio
        if self.audio_data is not None:
            audio_filename = f"{word}_{example_idx:02d}.wav"
            audio_path = os.path.join(audio_dir, word, audio_filename)
            sf.write(audio_path, self.audio_data, audio_fs)
        
        # Sauvegarder les régions des lèvres
        lip_count = 0
        for i, lip_region in enumerate(self.lip_regions):
            lip_filename = f"{word}_{example_idx:02d}_{i+1:03d}.png"
            lip_path = os.path.join(lip_dir, word, lip_filename)
            cv2.imwrite(lip_path, lip_region)
            lip_count += 1
        
        # Afficher un message dans la zone de statut au lieu d'une boîte de dialogue
        self.status_message.set(f"Enregistrement sauvegardé. {lip_count} images de lèvres sauvegardées.")
        
        # Nettoyer les fichiers temporaires
        self.cleanup_temp_files()
        
        # Passer à l'exemple suivant
        if example_idx < examples_per_word:
            self.current_example.set(example_idx + 1)
        else:
            # Passer au mot suivant si tous les exemples sont faits
            current_word_idx = target_words.index(word)
            if current_word_idx < len(target_words) - 1:
                self.current_word.set(target_words[current_word_idx + 1])
                self.current_example.set(1)
            else:
                self.status_message.set("Tous les mots et exemples ont été enregistrés!")
        
        # Réinitialiser l'interface
        self.video_quality.set("")
        self.countdown_value.set(3)
        self.lip_canvas.delete("all")
        self.replay_btn.configure(state="disabled")
    
    def reject_recording(self):
        # Réinitialiser sans sauvegarder
        self.frames = []
        self.lip_regions = []
        self.audio_data = None
        self.video_quality.set("")
        self.countdown_value.set(3)
        self.lip_canvas.delete("all")
        self.replay_btn.configure(state="disabled")
        self.status_message.set("Enregistrement rejeté.")
        
        # Nettoyer les fichiers temporaires
        self.cleanup_temp_files()
    
    def on_closing(self):
        # Nettoyer les fichiers temporaires
        self.cleanup_temp_files()
        
        # Libérer les ressources
        if self.cap is not None:
            self.cap.release()
        
        # Fermer pygame
        pygame.mixer.quit()
        
        # Fermer l'application
        self.root.destroy()

In [5]:
if __name__ == "__main__":
    # Vérifier le détecteur
    if not os.path.exists(predictor_path):
        print("ERREUR: Le fichier de points de repère n'a pas été trouvé !")
        print("Téléchargez-le depuis http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")
        print("Puis décompressez-le et placez-le dans le dossier 'models'")
        exit(1)
    
    # Créer l'application
    root = tk.Tk()
    app = VSRDataCollector(root)
    root.mainloop()
